<h1>contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#setup" data-toc-modified-id="setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>setup</a></span></li><li><span><a href="#data" data-toc-modified-id="data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>data</a></span></li></ul></div>

# setup

In [1]:
import datetime
import gc
import json
import os
import sys
import pprint
import warnings
from io import BytesIO

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import folium
import geopandas as gpd
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import netCDF4 as nc
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
from folium.plugins import HeatMap
from IPython.display import display
from scipy.interpolate import griddata
from selenium import webdriver
from skimage.transform import resize
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import ParameterGrid, train_test_split
from sklearn.preprocessing import MinMaxScaler

mpl.rcParams.update(mpl.rcParamsDefault)
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
print(os.uname())
print()
print(sys.version)
print(sys.version_info)
print()
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

posix.uname_result(sysname='Linux', nodename='u22', release='6.2.0-36-generic', version='#37~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Oct  9 15:34:04 UTC 2', machine='x86_64')

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0)

2.0.0+cu117
11.7
8500
NVIDIA GeForce GTX 1660 Ti
True


In [3]:
sns.set_style("whitegrid")

plt.rcParams.update({
    'font.size'       : 9,
    'figure.figsize'  : (3.5, 2.8),
    'figure.dpi'      : 300,
    'savefig.dpi'     : 300,
    'text.usetex'     : True,
    'font.family'     : 'serif',
    'font.serif'      : ['Times New Roman'],
    'axes.labelsize'  : 9,
    'axes.titlesize'  : 9,
    'xtick.labelsize' : 8,
    'ytick.labelsize' : 8,
    'legend.fontsize' : 8,
    'lines.linewidth' : 1,
    'axes.linewidth'  : 1,
    'grid.linestyle'  : '--',
    'grid.linewidth'  : 0.5,
    'grid.alpha'      : 0.8,
})

sns.set_context(
    "paper", 
    rc={
        "font.size"      : 9,
        "axes.titlesize" : 9,
        "axes.labelsize" : 9, 
        'xtick.labelsize': 8,
        'ytick.labelsize': 8
    }
)

In [4]:
experiment_name = "wildfire_spread"
folder_path = f"../data/08_reporting/{experiment_name}"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

golden_ratio = (np.sqrt(5) - 1) / 2

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
BRAZIL_EXTENT = [-74, -34, -34, 6]
SAO_PAULO_EXTENT = [-53, -44, -25, -20]

In [6]:
MAX_LAT = 32.99499999641088
MIN_LAT = -55.98500000358912
MAX_LON = -33.009801806103724
MIN_LON = -119.98980180610373

MAX_LAT = BRAZIL_EXTENT[3]
MIN_LAT = BRAZIL_EXTENT[2]
MAX_LON = BRAZIL_EXTENT[1]
MIN_LON = BRAZIL_EXTENT[0]

# data

In [7]:
DATA = torch.load(f"../data/02_intermediate/wildfire_spread/wildfire_spread_tensor.pt")

In [8]:
DATA.shape

torch.Size([37, 5, 2400, 2400])

In [17]:
TARGET = DATA[:, 0, :, :]
FEATURES = DATA[:, 1:, :, :]

print("DATA")
print(TARGET.shape)
print(FEATURES.shape)

# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    FEATURES, TARGET, test_size=0.1, random_state=1
)

print()
print("TRAIN AND TEST SETS")
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


DATA
torch.Size([37, 2400, 2400])
torch.Size([37, 4, 2400, 2400])

TRAIN AND TEST SETS
torch.Size([33, 4, 2400, 2400])
torch.Size([4, 4, 2400, 2400])
torch.Size([33, 2400, 2400])
torch.Size([4, 2400, 2400])


# architecture

In [9]:
class WildfirePredictor(nn.Module):
    """
    WildfirePredictor
    """
    def __init__(self):
        super(WildfirePredictor, self).__init__()
        # CNN layers
        self.conv1 = nn.Conv2d(5, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.lstm = nn.LSTM(input_size=2400, hidden_size=100, num_layers=2, batch_first=True)

        self.fc = nn.Linear(100, 2400 * 2400)

    def forward(self, x):
        batch_size, timesteps, C, H, W = x.size()
        c_out = torch.empty(batch_size, timesteps, 32, H, W, device=x.device)
        for t in range(timesteps):
            x_t = self.pool(torch.relu(self.conv1(x[:, t])))
            x_t = self.pool(torch.relu(self.conv2(x_t)))

            c_out[:, t] = x_t.view(batch_size, 32, -1)

        r_out, (h_n, c_n) = self.lstm(c_out)

        r_out = r_out[:, -1] 
        r_out = self.fc(r_out)

        r_out = r_out.view(batch_size, 2400, 2400)

        return r_out